In [1]:
dtype = {'Census_InternalBatteryType_lion': 'int8',
 'Census_InternalBatteryType_#': 'int8',
 'Census_InternalBatteryType_li-i': 'int8',
 'Census_InternalBatteryType_lip': 'int8',
 'Census_InternalBatteryType_liio': 'int8',
 'Census_InternalBatteryType_li p': 'int8',
 'Census_InternalBatteryType_real': 'int8',
 'Census_InternalBatteryType_li': 'int8',
 'Census_InternalBatteryType_pbac': 'int8',
 'Census_InternalBatteryType_vbox': 'int8',
 'Census_InternalBatteryType_bq20': 'int8',
 'Census_InternalBatteryType_nimh': 'int8',
 'Census_InternalBatteryType_lgi0': 'int8',
 'Census_InternalBatteryType_batt': 'int8',
 'Census_InternalBatteryType_4cel': 'int8',
 'Census_InternalBatteryType_unkn': 'int8',
 'Census_InternalBatteryType_ithi': 'int8',
 'Census_InternalBatteryType_lipo': 'int8',
 'Census_InternalBatteryType_pad0': 'int8',
 'Census_InternalBatteryType_lhp0': 'int8',
 'Census_InternalBatteryType_a132': 'int8',
 'Census_InternalBatteryType_4lio': 'int8',
 'Census_InternalBatteryType_lio': 'int8',
 'Census_InternalBatteryType_virt': 'int8',
 'Census_InternalBatteryType_bad': 'int8',
 'Census_InternalBatteryType_lit': 'int8',
 'Census_InternalBatteryType_lipp': 'int8',
 'Census_InternalBatteryType_ram': 'int8',
 'Census_InternalBatteryType_ca48': 'int8',
 'Census_InternalBatteryType_p-sn': 'int8',
 'Census_OSInstallLanguageIdentifier_37.0': 'int8',
 'Census_OSInstallLanguageIdentifier_8.0': 'int8',
 'Census_OSInstallLanguageIdentifier_10.0': 'int8',
 'Census_OSInstallLanguageIdentifier_9.0': 'int8',
 'Census_OSInstallLanguageIdentifier_7.0': 'int8',
 'Census_OSInstallLanguageIdentifier_30.0': 'int8',
 'Census_OSInstallLanguageIdentifier_25.0': 'int8',
 'Census_OSInstallLanguageIdentifier_20.0': 'int8',
 'Census_OSInstallLanguageIdentifier_27.0': 'int8',
 'Census_OSInstallLanguageIdentifier_35.0': 'int8',
 'Census_OSInstallLanguageIdentifier_1.0': 'int8',
 'Census_OSInstallLanguageIdentifier_14.0': 'int8',
 'Census_OSInstallLanguageIdentifier_39.0': 'int8',
 'Census_OSInstallLanguageIdentifier_29.0': 'int8',
 'Census_OSInstallLanguageIdentifier_36.0': 'int8',
 'Census_OSInstallLanguageIdentifier_5.0': 'int8',
 'Census_OSInstallLanguageIdentifier_26.0': 'int8',
 'Census_OSInstallLanguageIdentifier_15.0': 'int8',
 'Census_OSInstallLanguageIdentifier_11.0': 'int8',
 'Census_OSInstallLanguageIdentifier_31.0': 'int8',
 'Census_OSInstallLanguageIdentifier_4.0': 'int8',
 'Census_OSInstallLanguageIdentifier_6.0': 'int8',
 'Census_OSInstallLanguageIdentifier_18.0': 'int8',
 'Census_OSInstallLanguageIdentifier_17.0': 'int8',
 'Census_OSInstallLanguageIdentifier_23.0': 'int8',
 'Census_OSInstallLanguageIdentifier_12.0': 'int8',
 'Census_OSInstallLanguageIdentifier_3.0': 'int8',
 'Census_OSInstallLanguageIdentifier_24.0': 'int8',
 'Census_OSInstallLanguageIdentifier_19.0': 'int8',
 'Census_OSInstallLanguageIdentifier_32.0': 'int8',
 'Census_OSInstallLanguageIdentifier_28.0': 'int8',
 'Census_OSInstallLanguageIdentifier_33.0': 'int8',
 'Census_OSInstallLanguageIdentifier_38.0': 'int8',
 'Census_OSInstallLanguageIdentifier_16.0': 'int8',
 'Census_OSInstallLanguageIdentifier_2.0': 'int8',
 'Census_OSInstallLanguageIdentifier_34.0': 'int8',
 'Census_OSInstallLanguageIdentifier_13.0': 'int8',
 'Census_OSInstallLanguageIdentifier_21.0': 'int8',
 'Census_OSInstallLanguageIdentifier_22.0': 'int8',
 'Census_PowerPlatformRoleName_Mobile': 'int8',
 'Census_PowerPlatformRoleName_Desktop': 'int8',
 'Census_PowerPlatformRoleName_Slate': 'int8',
 'Census_PowerPlatformRoleName_Workstation': 'int8',
 'Census_PowerPlatformRoleName_UNKNOWN': 'int8',
 'Census_PowerPlatformRoleName_SOHOServer': 'int8',
 'Census_PowerPlatformRoleName_EnterpriseServer': 'int8',
 'Census_PowerPlatformRoleName_AppliancePC': 'int8',
 'Census_PowerPlatformRoleName_PerformanceServer': 'int8',
 'Census_PrimaryDiskTypeName_HDD': 'int8',
 'Census_PrimaryDiskTypeName_SSD': 'int8',
 'Census_PrimaryDiskTypeName_Unspecified': 'int8',
 'Census_PrimaryDiskTypeName_UNKNOWN': 'int8',
 'Census_ProcessorManufacturerIdentifier_5.0': 'int8',
 'Census_ProcessorManufacturerIdentifier_1.0': 'int8',
 'Census_ProcessorManufacturerIdentifier_10.0': 'int8',
 'Census_ProcessorManufacturerIdentifier_3.0': 'int8',
 'OrganizationIdentifier_27.0': 'int8',
 'OrganizationIdentifier_18.0': 'int8',
 'OrganizationIdentifier_11.0': 'int8',
 'OrganizationIdentifier_22.0': 'int8',
 'OrganizationIdentifier_48.0': 'int8',
 'OrganizationIdentifier_46.0': 'int8',
 'OrganizationIdentifier_10.0': 'int8',
 'OrganizationIdentifier_37.0': 'int8',
 'OrganizationIdentifier_5.0': 'int8',
 'OrganizationIdentifier_50.0': 'int8',
 'OrganizationIdentifier_49.0': 'int8',
 'OrganizationIdentifier_52.0': 'int8',
 'OrganizationIdentifier_32.0': 'int8',
 'OrganizationIdentifier_51.0': 'int8',
 'OrganizationIdentifier_2.0': 'int8',
 'OrganizationIdentifier_33.0': 'int8',
 'OrganizationIdentifier_14.0': 'int8',
 'OrganizationIdentifier_20.0': 'int8',
 'OrganizationIdentifier_36.0': 'int8',
 'OrganizationIdentifier_6.0': 'int8',
 'OrganizationIdentifier_39.0': 'int8',
 'OrganizationIdentifier_4.0': 'int8',
 'OrganizationIdentifier_1.0': 'int8',
 'OrganizationIdentifier_28.0': 'int8',
 'OrganizationIdentifier_16.0': 'int8',
 'OrganizationIdentifier_40.0': 'int8',
 'OrganizationIdentifier_30.0': 'int8',
 'OrganizationIdentifier_43.0': 'int8',
 'OrganizationIdentifier_21.0': 'int8',
 'OrganizationIdentifier_47.0': 'int8',
 'OrganizationIdentifier_8.0': 'int8',
 'OrganizationIdentifier_3.0': 'int8',
 'OrganizationIdentifier_7.0': 'int8',
 'OrganizationIdentifier_45.0': 'int8',
 'OrganizationIdentifier_26.0': 'int8',
 'OrganizationIdentifier_25.0': 'int8',
 'OrganizationIdentifier_42.0': 'int8',
 'OrganizationIdentifier_35.0': 'int8',
 'OrganizationIdentifier_31.0': 'int8',
 'OrganizationIdentifier_41.0': 'int8',
 'OrganizationIdentifier_44.0': 'int8',
 'OrganizationIdentifier_29.0': 'int8',
 'OrganizationIdentifier_19.0': 'int8',
 'OrganizationIdentifier_23.0': 'int8',
 'AppVersion_4.18': 'int8',
 'AppVersion_4.14': 'int8',
 'AppVersion_4.13': 'int8',
 'AppVersion_4.11': 'int8',
 'AppVersion_4.12': 'int8',
 'AppVersion_4.10': 'int8',
 'AppVersion_4.16': 'int8',
 'AppVersion_4.9': 'int8',
 'AppVersion_4.8': 'int8',
 'AppVersion_4.5': 'int8',
 'AppVersion_4.6': 'int8',
 'AppVersion_4.4': 'int8',
 'AppVersion_4.7': 'int8',
 'AppVersion_4.17': 'int8',
 'AppVersion_4.15': 'int8',
 'AvSigVersion_1.273': 'int8',
 'AvSigVersion_1.275': 'int8',
 'AvSigVersion_1.271': 'int8',
 'AvSigVersion_1.263': 'int8',
 'AvSigVersion_1.261': 'int8',
 'AvSigVersion_1.257': 'int8',
 'AvSigVersion_1.269': 'int8',
 'AvSigVersion_1.267': 'int8',
 'AvSigVersion_1.277': 'int8',
 'AvSigVersion_1.231': 'int8',
 'AvSigVersion_1.247': 'int8',
 'AvSigVersion_1.237': 'int8',
 'AvSigVersion_1.259': 'int8',
 'AvSigVersion_1.251': 'int8',
 'AvSigVersion_1.229': 'int8',
 'AvSigVersion_1.249': 'int8',
 'AvSigVersion_1.239': 'int8',
 'AvSigVersion_1.245': 'int8',
 'AvSigVersion_1.253': 'int8',
 'AvSigVersion_1.243': 'int8',
 'AvSigVersion_1.265': 'int8',
 'AvSigVersion_1.233': 'int8',
 'AvSigVersion_1.235': 'int8',
 'AvSigVersion_1.225': 'int8',
 'AvSigVersion_1.241': 'int8',
 'AvSigVersion_1.255': 'int8',
 'AvSigVersion_1.223': 'int8',
 'AvSigVersion_1.227': 'int8',
 'AvSigVersion_1.221': 'int8',
 'AvSigVersion_1.213': 'int8',
 'AvSigVersion_0.0': 'int8',
 'AvSigVersion_1.199': 'int8',
 'AvSigVersion_1.207': 'int8',
 'AvSigVersion_1.197': 'int8',
 'AvSigVersion_1.203': 'int8',
 'AvSigVersion_1.177': 'int8',
 'AvSigVersion_1.2&#x17;3': 'int8',
 'AvSigVersion_1.167': 'int8',
 'Census_ActivationChannel_Retail': 'int8',
 'Census_ActivationChannel_OEM:DM': 'int8',
 'Census_ActivationChannel_OEM:NONSLP': 'int8',
 'Census_ActivationChannel_Volume:GVLK': 'int8',
 'Census_ActivationChannel_Volume:MAK': 'int8',
 'Census_ActivationChannel_Retail:TB:Eval': 'int8',
 'Census_DeviceFamily_Windows.Desktop': 'int8',
 'Census_DeviceFamily_Windows.Server': 'int8',
 'Census_DeviceFamily_Windows': 'int8',
 'Census_FlightRing_Retail': 'int8',
 'Census_FlightRing_Unknown': 'int8',
 'Census_FlightRing_WIS': 'int8',
 'Census_FlightRing_NOT_SET': 'int8',
 'Census_FlightRing_RP': 'int8',
 'Census_FlightRing_WIF': 'int8',
 'Census_FlightRing_Disabled': 'int8',
 'Census_GenuineStateName_IS_GENUINE': 'int8',
 'Census_GenuineStateName_INVALID_LICENSE': 'int8',
 'Census_GenuineStateName_OFFLINE': 'int8',
 'Census_GenuineStateName_UNKNOWN': 'int8',
 'Census_MDC2FormFactor_Notebook': 'int8',
 'Census_MDC2FormFactor_Convertible': 'int8',
 'Census_MDC2FormFactor_Desktop': 'int8',
 'Census_MDC2FormFactor_AllInOne': 'int8',
 'Census_MDC2FormFactor_SmallServer': 'int8',
 'Census_MDC2FormFactor_Detachable': 'int8',
 'Census_MDC2FormFactor_LargeTablet': 'int8',
 'Census_MDC2FormFactor_SmallTablet': 'int8',
 'Census_MDC2FormFactor_PCOther': 'int8',
 'Census_MDC2FormFactor_MediumServer': 'int8',
 'Census_MDC2FormFactor_LargeServer': 'int8',
 'Census_MDC2FormFactor_ServerOther': 'int8',
 'Census_OSArchitecture_amd64': 'int8',
 'Census_OSArchitecture_x86': 'int8',
 'Census_OSArchitecture_arm64': 'int8',
 'Census_OSBranch_rs4_release': 'int8',
 'Census_OSBranch_rs1_release': 'int8',
 'Census_OSBranch_rs3_release': 'int8',
 'Census_OSBranch_rs2_release': 'int8',
 'Census_OSBranch_rs3_release_svc_escrow': 'int8',
 'Census_OSBranch_th2_release': 'int8',
 'Census_OSBranch_th2_release_sec': 'int8',
 'Census_OSBranch_rs5_release': 'int8',
 'Census_OSBranch_th1_st1': 'int8',
 'Census_OSBranch_th1': 'int8',
 'Census_OSBranch_rs_prerelease': 'int8',
 'Census_OSBranch_rs3_release_svc_escrow_im': 'int8',
 'Census_OSBranch_rs_prerelease_flt': 'int8',
 'Census_OSBranch_winblue_ltsb_escrow': 'int8',
 'Census_OSBranch_rs5_release_sigma': 'int8',
 'Census_OSBranch_rs5_release_sigma_dev': 'int8',
 'Census_OSBuildNumber_17134': 'int8',
 'Census_OSBuildNumber_14393': 'int8',
 'Census_OSBuildNumber_16299': 'int8',
 'Census_OSBuildNumber_15063': 'int8',
 'Census_OSBuildNumber_10586': 'int8',
 'Census_OSBuildNumber_17733': 'int8',
 'Census_OSBuildNumber_10240': 'int8',
 'Census_OSBuildNumber_17755': 'int8',
 'Census_OSBuildNumber_17746': 'int8',
 'Census_OSBuildNumber_17639': 'int8',
 'Census_OSBuildNumber_17754': 'int8',
 'Census_OSBuildNumber_17751': 'int8',
 'Census_OSBuildNumber_17758': 'int8',
 'Census_OSBuildNumber_17666': 'int8',
 'Census_OSBuildNumber_17738': 'int8',
 'Census_OSBuildNumber_17763': 'int8',
 'Census_OSBuildNumber_17735': 'int8',
 'Census_OSBuildNumber_17744': 'int8',
 'Census_OSBuildNumber_17133': 'int8',
 'Census_OSBuildNumber_17692': 'int8',
 'Census_OSBuildNumber_17760': 'int8',
 'Census_OSBuildNumber_15055': 'int8',
 'Census_OSBuildNumber_17650': 'int8',
 'Census_OSBuildNumber_17672': 'int8',
 'Census_OSBuildNumber_17686': 'int8',
 'Census_OSBuildNumber_17741': 'int8',
 'Census_OSBuildNumber_17677': 'int8',
 'Census_OSBuildNumber_18242': 'int8',
 'Census_OSBuildNumber_14946': 'int8',
 'Census_OSBuildNumber_17723': 'int8',
 'Census_OSBuildNumber_17713': 'int8',
 'Census_OSBuildNumber_14474': 'int8',
 'Census_OSBuildNumber_18237': 'int8',
 'Census_OSBuildNumber_10565': 'int8',
 'Census_OSBuildNumber_17115': 'int8',
 'Census_OSBuildNumber_17618': 'int8',
 'Census_OSBuildNumber_18234': 'int8',
 'Census_OSBuildNumber_17682': 'int8',
 'Census_OSBuildNumber_14366': 'int8',
 'Census_OSBuildNumber_17711': 'int8',
 'Census_OSBuildNumber_14390': 'int8',
 'Census_OSBuildNumber_17661': 'int8',
 'Census_OSBuildNumber_14931': 'int8',
 'Census_OSBuildNumber_14986': 'int8',
 'Census_OSBuildNumber_14421': 'int8',
 'Census_OSBuildNumber_14466': 'int8',
 'Census_OSBuildNumber_9600': 'int8',
 'Census_OSBuildNumber_17747': 'int8',
 'Census_OSBuildNumber_14951': 'int8',
 'Census_OSBuildNumber_17627': 'int8',
 'Census_OSBuildNumber_14915': 'int8',
 'Census_OSBuildNumber_14332': 'int8',
 'Census_OSBuildNumber_17730': 'int8',
 'Census_OSBuildNumber_7601': 'int8',
 'Census_OSBuildNumber_14971': 'int8',
 'Census_OSBuildNumber_17025': 'int8',
 'Census_OSBuildNumber_17655': 'int8',
 'Census_OSBuildNumber_15048': 'int8',
 'Census_OSBuildNumber_17634': 'int8',
 'Census_OSBuildNumber_17074': 'int8',
 'Census_OSBuildNumber_14936': 'int8',
 'Census_OSBuildNumber_14295': 'int8',
 'Census_OSBuildNumber_16193': 'int8',
 'Census_OSBuildNumber_17761': 'int8',
 'Census_OSEdition_Professional': 'int8',
 'Census_OSEdition_Core': 'int8',
 'Census_OSEdition_CoreSingleLanguage': 'int8',
 'Census_OSEdition_CoreCountrySpecific': 'int8',
 'Census_OSEdition_ServerStandard': 'int8',
 'Census_OSEdition_ProfessionalEducation': 'int8',
 'Census_OSEdition_Enterprise': 'int8',
 'Census_OSEdition_ProfessionalN': 'int8',
 'Census_OSEdition_EnterpriseS': 'int8',
 'Census_OSEdition_ServerStandardEval': 'int8',
 'Census_OSEdition_Education': 'int8',
 'Census_OSEdition_CoreN': 'int8',
 'Census_OSEdition_Cloud': 'int8',
 'Census_OSEdition_ServerDatacenterEval': 'int8',
 'Census_OSEdition_EducationN': 'int8',
 'Census_OSEdition_EnterpriseSN': 'int8',
 'Census_OSEdition_ProfessionalWorkstation': 'int8',
 'Census_OSEdition_ServerSolution': 'int8',
 'Census_OSEdition_EnterpriseN': 'int8',
 'Census_OSEdition_ProfessionalEducationN': 'int8',
 'Census_OSEdition_CloudN': 'int8',
 'Census_OSEdition_Home': 'int8',
 'Census_OSEdition_ProfessionalCountrySpecific': 'int8',
 'Census_OSInstallTypeName_Clean': 'int8',
 'Census_OSInstallTypeName_Update': 'int8',
 'Census_OSInstallTypeName_Upgrade': 'int8',
 'Census_OSInstallTypeName_Reset': 'int8',
 'Census_OSInstallTypeName_IBSClean': 'int8',
 'Census_OSInstallTypeName_UUPUpgrade': 'int8',
 'Census_OSInstallTypeName_Other': 'int8',
 'Census_OSInstallTypeName_Refresh': 'int8',
 'Census_OSInstallTypeName_CleanPCRefresh': 'int8',
 'Census_OSSkuName_PROFESSIONAL': 'int8',
 'Census_OSSkuName_CORE': 'int8',
 'Census_OSSkuName_CORE_SINGLELANGUAGE': 'int8',
 'Census_OSSkuName_CORE_COUNTRYSPECIFIC': 'int8',
 'Census_OSSkuName_STANDARD_SERVER': 'int8',
 'Census_OSSkuName_ENTERPRISE': 'int8',
 'Census_OSSkuName_PROFESSIONAL_N': 'int8',
 'Census_OSSkuName_ENTERPRISE_S': 'int8',
 'Census_OSSkuName_STANDARD_EVALUATION_SERVER': 'int8',
 'Census_OSSkuName_EDUCATION': 'int8',
 'Census_OSSkuName_CORE_N': 'int8',
 'Census_OSSkuName_CLOUD': 'int8',
 'Census_OSSkuName_DATACENTER_EVALUATION_SERVER': 'int8',
 'Census_OSSkuName_EDUCATION_N': 'int8',
 'Census_OSSkuName_ENTERPRISE_S_N': 'int8',
 'Census_OSSkuName_PRO_WORKSTATION': 'int8',
 'Census_OSSkuName_SB_SOLUTION_SERVER': 'int8',
 'Census_OSSkuName_ENTERPRISE_N': 'int8',
 'Census_OSSkuName_ENTERPRISEG': 'int8',
 'Census_OSSkuName_UNLICENSED': 'int8',
 'Census_OSSkuName_PRO_CHINA': 'int8',
 'Census_OSUILocaleIdentifier_158': 'int8',
 'Census_OSUILocaleIdentifier_31': 'int8',
 'Census_OSUILocaleIdentifier_35': 'int8',
 'Census_OSUILocaleIdentifier_34': 'int8',
 'Census_OSUILocaleIdentifier_30': 'int8',
 'Census_OSUILocaleIdentifier_128': 'int8',
 'Census_OSUILocaleIdentifier_115': 'int8',
 'Census_OSUILocaleIdentifier_83': 'int8',
 'Census_OSUILocaleIdentifier_120': 'int8',
 'Census_OSUILocaleIdentifier_148': 'int8',
 'Census_OSUILocaleIdentifier_5': 'int8',
 'Census_OSUILocaleIdentifier_49': 'int8',
 'Census_OSUILocaleIdentifier_160': 'int8',
 'Census_OSUILocaleIdentifier_125': 'int8',
 'Census_OSUILocaleIdentifier_151': 'int8',
 'Census_OSUILocaleIdentifier_26': 'int8',
 'Census_OSUILocaleIdentifier_119': 'int8',
 'Census_OSUILocaleIdentifier_58': 'int8',
 'Census_OSUILocaleIdentifier_38': 'int8',
 'Census_OSUILocaleIdentifier_130': 'int8',
 'Census_OSUILocaleIdentifier_24': 'int8',
 'Census_OSUILocaleIdentifier_28': 'int8',
 'Census_OSUILocaleIdentifier_72': 'int8',
 'Census_OSUILocaleIdentifier_64': 'int8',
 'Census_OSUILocaleIdentifier_105': 'int8',
 'Census_OSUILocaleIdentifier_44': 'int8',
 'Census_OSUILocaleIdentifier_20': 'int8',
 'Census_OSUILocaleIdentifier_109': 'int8',
 'Census_OSUILocaleIdentifier_74': 'int8',
 'Census_OSUILocaleIdentifier_136': 'int8',
 'Census_OSUILocaleIdentifier_123': 'int8',
 'Census_OSUILocaleIdentifier_140': 'int8',
 'Census_OSUILocaleIdentifier_159': 'int8',
 'Census_OSUILocaleIdentifier_62': 'int8',
 'Census_OSUILocaleIdentifier_11': 'int8',
 'Census_OSUILocaleIdentifier_145': 'int8',
 'Census_OSUILocaleIdentifier_48': 'int8',
 'Census_OSUILocaleIdentifier_156': 'int8',
 'Census_OSUILocaleIdentifier_90': 'int8',
 'Census_OSUILocaleIdentifier_51': 'int8',
 'Census_OSUILocaleIdentifier_92': 'int8',
 'Census_OSUILocaleIdentifier_68': 'int8',
 'Census_OSUILocaleIdentifier_32': 'int8',
 'Census_OSUILocaleIdentifier_76': 'int8',
 'Census_OSUILocaleIdentifier_17': 'int8',
 'Census_OSUILocaleIdentifier_91': 'int8',
 'Census_OSUILocaleIdentifier_63': 'int8',
 'Census_OSUILocaleIdentifier_54': 'int8',
 'Census_OSUILocaleIdentifier_15': 'int8',
 'Census_OSUILocaleIdentifier_73': 'int8',
 'Census_OSUILocaleIdentifier_29': 'int8',
 'Census_OSUILocaleIdentifier_152': 'int8',
 'Census_OSUILocaleIdentifier_70': 'int8',
 'Census_OSUILocaleIdentifier_124': 'int8',
 'Census_OSUILocaleIdentifier_95': 'int8',
 'Census_OSUILocaleIdentifier_21': 'int8',
 'Census_OSUILocaleIdentifier_19': 'int8',
 'Census_OSUILocaleIdentifier_40': 'int8',
 'Census_OSUILocaleIdentifier_146': 'int8',
 'Census_OSUILocaleIdentifier_116': 'int8',
 'Census_OSUILocaleIdentifier_80': 'int8',
 'Census_OSUILocaleIdentifier_161': 'int8',
 'Census_OSUILocaleIdentifier_37': 'int8',
 'Census_OSUILocaleIdentifier_42': 'int8',
 'Census_OSUILocaleIdentifier_59': 'int8',
 'Census_OSUILocaleIdentifier_126': 'int8',
 'Census_OSUILocaleIdentifier_121': 'int8',
 'Census_OSUILocaleIdentifier_12': 'int8',
 'Census_OSUILocaleIdentifier_36': 'int8',
 'Census_OSUILocaleIdentifier_39': 'int8',
 'Census_OSUILocaleIdentifier_50': 'int8',
 'Census_OSUILocaleIdentifier_157': 'int8',
 'Census_OSUILocaleIdentifier_6': 'int8',
 'Census_OSUILocaleIdentifier_25': 'int8',
 'Census_OSUILocaleIdentifier_33': 'int8',
 'Census_OSUILocaleIdentifier_93': 'int8',
 'Census_OSUILocaleIdentifier_131': 'int8',
 'Census_OSUILocaleIdentifier_132': 'int8',
 'Census_OSUILocaleIdentifier_89': 'int8',
 'Census_OSUILocaleIdentifier_81': 'int8',
 'Census_OSUILocaleIdentifier_122': 'int8',
 'Census_OSUILocaleIdentifier_129': 'int8',
 'Census_OSUILocaleIdentifier_102': 'int8',
 'Census_OSUILocaleIdentifier_110': 'int8',
 'Census_OSUILocaleIdentifier_7': 'int8',
 'Census_OSUILocaleIdentifier_139': 'int8',
 'Census_OSUILocaleIdentifier_69': 'int8',
 'Census_OSUILocaleIdentifier_138': 'int8',
 'Census_OSUILocaleIdentifier_41': 'int8',
 'Census_OSUILocaleIdentifier_60': 'int8',
 'Census_OSUILocaleIdentifier_98': 'int8',
 'Census_OSUILocaleIdentifier_84': 'int8',
 'Census_OSUILocaleIdentifier_8': 'int8',
 'Census_OSUILocaleIdentifier_99': 'int8',
 'Census_OSVersion_10.0.17134': 'int8',
 'Census_OSVersion_10.0.14393': 'int8',
 'Census_OSVersion_10.0.16299': 'int8',
 'Census_OSVersion_10.0.15063': 'int8',
 'Census_OSVersion_10.0.10586': 'int8',
 'Census_OSVersion_10.0.17733': 'int8',
 'Census_OSVersion_10.0.10240': 'int8',
 'Census_OSVersion_10.0.17755': 'int8',
 'Census_OSVersion_10.0.17746': 'int8',
 'Census_OSVersion_10.0.17639': 'int8',
 'Census_OSVersion_10.0.17754': 'int8',
 'Census_OSVersion_10.0.17751': 'int8',
 'Census_OSVersion_10.0.17758': 'int8',
 'Census_OSVersion_10.0.17666': 'int8',
 'Census_OSVersion_10.0.17738': 'int8',
 'Census_OSVersion_10.0.17763': 'int8',
 'Census_OSVersion_10.0.17735': 'int8',
 'Census_OSVersion_10.0.17744': 'int8',
 'Census_OSVersion_10.0.17133': 'int8',
 'Census_OSVersion_10.0.17692': 'int8',
 'Census_OSVersion_10.0.17760': 'int8',
 'Census_OSVersion_10.0.15055': 'int8',
 'Census_OSVersion_10.0.17650': 'int8',
 'Census_OSVersion_10.0.17672': 'int8',
 'Census_OSVersion_10.0.17686': 'int8',
 'Census_OSVersion_10.0.17741': 'int8',
 'Census_OSVersion_10.0.17677': 'int8',
 'Census_OSVersion_10.0.18242': 'int8',
 'Census_OSVersion_10.0.14946': 'int8',
 'Census_OSVersion_10.0.17723': 'int8',
 'Census_OSVersion_10.0.17713': 'int8',
 'Census_OSVersion_10.0.14474': 'int8',
 'Census_OSVersion_10.0.18237': 'int8',
 'Census_OSVersion_10.0.10565': 'int8',
 'Census_OSVersion_10.0.17115': 'int8',
 'Census_OSVersion_10.0.17618': 'int8',
 'Census_OSVersion_10.0.18234': 'int8',
 'Census_OSVersion_10.0.17682': 'int8',
 'Census_OSVersion_10.0.14366': 'int8',
 'Census_OSVersion_10.0.17711': 'int8',
 'Census_OSVersion_10.0.14390': 'int8',
 'Census_OSVersion_10.0.17661': 'int8',
 'Census_OSVersion_10.0.14931': 'int8',
 'Census_OSVersion_10.0.14986': 'int8',
 'Census_OSVersion_10.0.14421': 'int8',
 'Census_OSVersion_10.0.14466': 'int8',
 'Census_OSVersion_6.3.9600': 'int8',
 'Census_OSVersion_10.0.17747': 'int8',
 'Census_OSVersion_10.0.14951': 'int8',
 'Census_OSVersion_10.0.17627': 'int8',
 'Census_OSVersion_10.0.14915': 'int8',
 'Census_OSVersion_10.0.14332': 'int8',
 'Census_OSVersion_10.0.17730': 'int8',
 'Census_OSVersion_10.0.7601': 'int8',
 'Census_OSVersion_10.0.14971': 'int8',
 'Census_OSVersion_10.0.17025': 'int8',
 'Census_OSVersion_10.0.17655': 'int8',
 'Census_OSVersion_10.0.15048': 'int8',
 'Census_OSVersion_10.0.17634': 'int8',
 'Census_OSVersion_10.0.17074': 'int8',
 'Census_OSVersion_10.0.14936': 'int8',
 'Census_OSVersion_10.0.14295': 'int8',
 'Census_OSVersion_10.0.16193': 'int8',
 'Census_OSVersion_10.0.17761': 'int8',
 'Census_OSWUAutoUpdateOptionsName_FullAuto': 'int8',
 'Census_OSWUAutoUpdateOptionsName_Notify': 'int8',
 'Census_OSWUAutoUpdateOptionsName_UNKNOWN': 'int8',
 'Census_OSWUAutoUpdateOptionsName_AutoInstallAndRebootAtMaintenanceTime': 'int8',
 'Census_OSWUAutoUpdateOptionsName_DownloadNotify': 'int8',
 'Census_OSWUAutoUpdateOptionsName_Off': 'int8',
 'EngineVersion_1.1.15100.1': 'int8',
 'EngineVersion_1.1.15200.1': 'int8',
 'EngineVersion_1.1.15000.2': 'int8',
 'EngineVersion_1.1.14600.4': 'int8',
 'EngineVersion_1.1.14500.5': 'int8',
 'EngineVersion_1.1.14306.0': 'int8',
 'EngineVersion_1.1.14901.4': 'int8',
 'EngineVersion_1.1.14800.3': 'int8',
 'EngineVersion_1.1.15300.5': 'int8',
 'EngineVersion_1.1.15300.6': 'int8',
 'EngineVersion_1.1.13202.0': 'int8',
 'EngineVersion_1.1.13903.0': 'int8',
 'EngineVersion_1.1.13504.0': 'int8',
 'EngineVersion_1.1.14700.3': 'int8',
 'EngineVersion_1.1.14405.2': 'int8',
 'EngineVersion_1.1.14104.0': 'int8',
 'EngineVersion_1.1.13103.0': 'int8',
 'EngineVersion_1.1.14003.0': 'int8',
 'EngineVersion_1.1.13601.0': 'int8',
 'EngineVersion_1.1.13804.0': 'int8',
 'EngineVersion_1.1.14202.0': 'int8',
 'EngineVersion_1.1.13704.0': 'int8',
 'EngineVersion_1.1.14700.5': 'int8',
 'EngineVersion_1.1.14303.0': 'int8',
 'EngineVersion_1.1.14901.3': 'int8',
 'EngineVersion_1.1.13303.0': 'int8',
 'EngineVersion_1.1.13407.0': 'int8',
 'EngineVersion_1.1.14700.4': 'int8',
 'EngineVersion_1.1.12902.0': 'int8',
 'EngineVersion_1.1.13701.0': 'int8',
 'EngineVersion_1.1.14305.0': 'int8',
 'EngineVersion_1.1.13000.0': 'int8',
 'EngineVersion_1.1.14500.2': 'int8',
 'EngineVersion_1.1.12805.0': 'int8',
 'EngineVersion_1.1.15000.1': 'int8',
 'EngineVersion_1.1.14002.0': 'int8',
 'EngineVersion_1.1.14800.1': 'int8',
 'EngineVersion_1.1.14102.0': 'int8',
 'EngineVersion_1.1.12400.0': 'int8',
 'EngineVersion_1.1.14201.0': 'int8',
 'EngineVersion_1.1.14103.0': 'int8',
 'EngineVersion_1.1.12804.0': 'int8',
 'EngineVersion_1.1.13406.0': 'int8',
 'EngineVersion_1.1.11701.0': 'int8',
 'EngineVersion_1.1.13802.0': 'int8',
 'EngineVersion_1.1.12101.0': 'int8',
 'EngineVersion_1.1.13902.0': 'int8',
 'EngineVersion_1.1.14001.0': 'int8',
 'EngineVersion_1.1.11602.0': 'int8',
 'EngineVersion_1.1.11903.0': 'int8',
 'EngineVersion_1.1.13803.0': 'int8',
 'EngineVersion_1.1.10701.0': 'int8',
 'EngineVersion_1.1.10302.0': 'int8',
 'OsBuild_17134': 'int8',
 'OsBuild_14393': 'int8',
 'OsBuild_16299': 'int8',
 'OsBuild_15063': 'int8',
 'OsBuild_9600': 'int8',
 'OsBuild_10586': 'int8',
 'OsBuild_17733': 'int8',
 'OsBuild_10240': 'int8',
 'OsBuild_7601': 'int8',
 'OsBuild_17755': 'int8',
 'OsBuild_17746': 'int8',
 'OsBuild_17639': 'int8',
 'OsBuild_17754': 'int8',
 'OsBuild_17751': 'int8',
 'OsBuild_17760': 'int8',
 'OsBuild_17666': 'int8',
 'OsBuild_17738': 'int8',
 'OsBuild_17763': 'int8',
 'OsBuild_17735': 'int8',
 'OsBuild_17744': 'int8',
 'OsBuild_17692': 'int8',
 'OsBuild_17713': 'int8',
 'OsBuild_17758': 'int8',
 'OsBuild_17650': 'int8',
 'OsBuild_17672': 'int8',
 'OsBuild_17686': 'int8',
 'OsBuild_17677': 'int8',
 'OsBuild_18242': 'int8',
 'OsBuild_7600': 'int8',
 'OsBuild_17741': 'int8',
 'OsBuild_18237': 'int8',
 'OsBuild_18234': 'int8',
 'OsBuild_17682': 'int8',
 'OsBuild_17661': 'int8',
 'OsBuild_17730': 'int8',
 'OsBuild_17723': 'int8',
 'OsBuild_17711': 'int8',
 'OsBuild_17747': 'int8',
 'OsBuild_17627': 'int8',
 'OsBuild_18219': 'int8',
 'OsBuild_17704': 'int8',
 'OsBuild_17618': 'int8',
 'OsBuild_17655': 'int8',
 'OsBuild_17734': 'int8',
 'OsBuild_17634': 'int8',
 'OsBuild_17728': 'int8',
 'OsBuild_18214': 'int8',
 'OsBuild_17643': 'int8',
 'OsBuild_17761': 'int8',
 'OsPlatformSubRelease_rs4': 'int8',
 'OsPlatformSubRelease_rs1': 'int8',
 'OsPlatformSubRelease_rs3': 'int8',
 'OsPlatformSubRelease_rs2': 'int8',
 'OsPlatformSubRelease_windows8.1': 'int8',
 'OsPlatformSubRelease_th2': 'int8',
 'OsPlatformSubRelease_prers5': 'int8',
 'OsPlatformSubRelease_th1': 'int8',
 'OsPlatformSubRelease_windows7': 'int8',
 'OsSuite_256': 'int8',
 'OsSuite_768': 'int8',
 'OsSuite_272': 'int8',
 'OsSuite_16': 'int8',
 'OsSuite_400': 'int8',
 'OsSuite_274': 'int8',
 'OsSuite_305': 'int8',
 'OsSuite_144': 'int8',
 'OsSuite_784': 'int8',
 'OsSuite_49': 'int8',
 'OsSuite_307': 'int8',
 'OsVer_10.0.0.0': 'int8',
 'OsVer_6.3.0.0': 'int8',
 'OsVer_6.1.1.0': 'int8',
 'OsVer_10.0.3.0': 'int8',
 'OsVer_6.1.4.0': 'int8',
 'OsVer_6.1.0.0': 'int8',
 'OsVer_6.1.2.0': 'int8',
 'OsVer_6.3.3.0': 'int8',
 'OsVer_10.0.1.0': 'int8',
 'OsVer_10.0.3.80': 'int8',
 'OsVer_10.0.8.0': 'int8',
 'OsVer_10.0.16.36': 'int8',
 'OsVer_10.0.32.72': 'int8',
 'OsVer_10.0.5.18': 'int8',
 'OsVer_10.0.5.0': 'int8',
 'OsVer_6.3.32.72': 'int8',
 'OsVer_6.1.3.0': 'int8',
 'OsVer_6.1.7.0': 'int8',
 'OsVer_6.3.4.0': 'int8',
 'OsVer_10.0.0.22': 'int8',
 'OsVer_6.3.1.0': 'int8',
 'OsVer_10.0.4.0': 'int8',
 'OsVer_6.3.0.112': 'int8',
 'Platform_windows10': 'int8',
 'Platform_windows8': 'int8',
 'Platform_windows2016': 'int8',
 'Platform_windows7': 'int8',
 'Processor_x64': 'int8',
 'Processor_x86': 'int8',
 'Processor_arm64': 'int8',
 'ProductName_win8defender': 'int8',
 'ProductName_mse': 'int8',
 'ProductName_mseprerelease': 'int8',
 'ProductName_fep': 'int8',
 'SkuEdition_Pro': 'int8',
 'SkuEdition_Home': 'int8',
 'SkuEdition_Invalid': 'int8',
 'SkuEdition_Enterprise': 'int8',
 'SkuEdition_Enterprise LTSB': 'int8',
 'SkuEdition_Server': 'int8',
 'SkuEdition_Education': 'int8',
 'SkuEdition_Cloud': 'int8',
 'RtpStateBitfield_7.0': 'int8',
 'RtpStateBitfield_0.0': 'int8',
 'RtpStateBitfield_5.0': 'int8',
 'RtpStateBitfield_8.0': 'int8',
 'RtpStateBitfield_3.0': 'int8',
 'RtpStateBitfield_1.0': 'int8',
 'RtpStateBitfield_35.0': 'int8',
 'IsBeta': 'int8',
 'IsSxsPassiveMode': 'int8',
 'HasTpm': 'int8',
 'IsProtected': 'int8',
 'AutoSampleOptIn': 'int8',
 'SMode': 'int8',
 'Firewall': 'int8',
 'UacLuaenable': 'int8',
 'Census_HasOpticalDiskDrive': 'int8',
 'Census_IsPortableOperatingSystem': 'int8',
 'Census_IsFlightsDisabled': 'int8',
 'Census_ThresholdOptIn': 'int8',
 'Census_IsSecureBootEnabled': 'int8',
 'Census_IsVirtualDevice': 'int8',
 'Census_IsTouchEnabled': 'int8',
 'Census_IsPenCapable': 'int8',
 'Census_IsAlwaysOnAlwaysConnectedCapable': 'int8',
 'HasDetections': 'int8'
 }

In [2]:
from sklearn.model_selection import ParameterSampler
import copy
import gc
from sklearn.metrics import roc_curve, auc
def hyper_parameter_optimization(params, modeloG, testX, testY, trainX, trainY):
    lista_parametros = list(ParameterSampler(params, n_iter=2))
    for elem in lista_parametros:
        altera = False
        for key, value in elem.items():
            if key == 'solver':
                if value in ['newton-cg','lbfgs','sag']:
                    altera = True
        if altera == True:
            elem['penalty'] = 'l2'
    tabelaFinal = []
    for param in lista_parametros:
        print(param)
        modelo = copy.copy(modeloG)
        modelo.set_params(**param)
        print(modelo.get_params())
        modelo.fit(trainX,trainY)
        pred = modelo.predict(testX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(testY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        elem = param.copy()
        elem['score'] = score
        tabelaFinal.append(elem)
        del modelo
        gc.collect()
    return tabelaFinal

In [3]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('encoding/final_sembat.csv',low_memory=False,chunksize=10000,dtype=dtype):
#     if i == 55:
#         break
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)
#aux = pd.read_csv('encoding/predict.csv',low_memory=False,dtype=dtype)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [4]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
trainX = auxPred.loc[:499999, auxPred.columns != 'HasDetections']
valX = auxPred.loc[500000:549999, auxPred.columns != 'HasDetections']
testX = X[125:]
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']
testY = y[125:]
print(trainX.shape)
print(valX.shape)
print(trainY.shape)
print(valY.shape)
# parametros = {'C':[0.001,0.01,0.1,1],'penalty':['l1','l2'],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
# m = LogisticRegression(max_iter=10000)
# tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)


(500000, 707)
(50000, 707)
(500000,)
(50000,)


In [11]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [1, 2, 4, 8, 16, 32, 64]#, 100, 200]
parametros = {'n_estimators':n_estimators}
m = RandomForestClassifier()
tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 8, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_

In [12]:
import pandas as pd
tab = pd.DataFrame(tabela)
tab.sort_values(by='score',ascending=False)

,n_estimators,score
3,32,0.652372
2,16,0.643802
1,8,0.629600
0,1,0.582084


In [36]:
import numpy as np
min_samples_splits = np.linspace(0.1, 0.3, 5, endpoint=True)
min_samples_splits

array([0.1 , 0.15, 0.2 , 0.25, 0.3 ])

In [37]:
min_samples_leafs = np.linspace(0, 0.2, 5, endpoint=True)
min_samples_leafs[0] = 0.00001
list(min_samples_leafs)

[1e-05, 0.05, 0.1, 0.15000000000000002, 0.2]

In [5]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [32,100]#[1, 2, 4, 8, 16, 32]#, 64]#, 100, 200]
parametros = {'n_estimators':n_estimators}#,'min_samples_leaf':list(min_samples_leafs),
             # 'min_samples_split':list(min_samples_splits)}
m = RandomForestClassifier()
tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

{'n_estimators': 32}
{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 32, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'n_estimators': 100}
{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [6]:
import pandas as pd
tab = pd.DataFrame(tabela)
tab.sort_values(by='score',ascending=False)

,n_estimators,score
1,100,0.659960
0,32,0.651811
